In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = PhysNet()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss=np_loss)
model(np.random.random((4, 128, 32, 32, 3)));
model.summary()

Model: "phys_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (4, 128, 32, 32, 16)   │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (4, 128, 16, 16, 32)   │        13,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 128, 16, 16, 64)   │        55,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_9 (Sequential)       │ (4, 64, 2, 2, 64)      │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_10 (Sequential)      │ (4, 128, 2, 2, 64)     │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_9 (Conv3D)               │ (4, 128, 1, 1, 1)      │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (4, 128, 16, 16, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (4, 64, 8, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling3d               │ (4, 128, 1, 1, 64)     │             0 │
│ (AveragePooling3D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (4, 128)               │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,825 (2.94 MB)

 Trainable params: 768,577 (2.93 MB)

 Non-trainable params: 1,248 (4.88 KB)

In [3]:
batch_size = 128

tape = "/root/ssd_cache/rppg_training_data/mmpd_128x32x32_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

training_set = training_set.apply_fn(compress_aug)

mmpd_128x32x32_all             datatape has been loaded.    10368 items total. fold=train selected
mmpd_128x32x32_all             datatape has been loaded.     2176 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/physnet_mmpd.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/physnet_mmpd.weights.h5')
model.load_weights('../weights/physnet_mmpd.weights.h5')

Training:   0%|          | 0/81 [00:00<?, ?it/s]

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.9965	Validation loss:     1	


Training:   0%|          | 0/81 [00:00<?, ?it/s]

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.3063	Validation loss:0.8894	


Training:   0%|          | 0/81 [00:00<?, ?it/s]

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch  20:  Training loss: 0.297	Validation loss:0.8952	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_MMPD_RLAP.h5', scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/PhysNet_MMPD_RLAP.h5', dropped='False')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,17.027±0.189,21.712±3.036,22.756±0.266,0.06731
Whole,8.738±0.476,11.234±3.637,11.665±0.655,0.18611


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_MMPD_COHFACE.h5', fps=30)
get_metrics('../results/PhysNet_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,19.227±0.357,24.556±4.471,28.446±0.569,0.0334
Whole,11.88±0.714,14.988±4.766,17.725±1.164,0.06158


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_MMPD_PURE.h5')
get_metrics('../results/PhysNet_MMPD_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,22.883±0.677,29.539±6.678,35.565±1.145,0.17901
Whole,18.465±1.682,22.535±8.565,28.69±2.748,0.40712


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_MMPD_UBFC.h5')
get_metrics('../results/PhysNet_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,17.971±0.907,27.419±7.439,17.491±0.838,0.25252
Whole,15.093±2.068,20.184±9.629,14.414±1.76,0.48952
